## Урок 5. SOAP

1. Запросить курсы любой валюты за указанный период. Сохранить полученные данные в базу данных Mongo

In [245]:
from zeep import Client
from pprint import pprint
from pymongo import MongoClient
import datetime 

def get_valute_code(valute):
    valutes = client.service.EnumValutes(0)
    list_valutes = valutes._value_1._value_1
    valute_code = ''
    for item in list_valutes:
        if item['EnumValutes']['VcharCode'] == valute:
              valute_code = item['EnumValutes']['Vcode']
    return valute_code

def get_curs_period(date1, date2, valute_code):
    dates = client.service.GetCursDynamic(date1, date2, valute_code)
    list_dates = dates._value_1._value_1
    curs_period = []
    for item in list_dates:
        curs_period.append({'Date': item['ValuteCursDynamic']['CursDate'],
                            'Curs': float(item['ValuteCursDynamic']['Vcurs'])})
    return curs_period

url='https://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?WSDL'
client = Client(url)
valute = 'USD'
valute_code = get_valute_code(valute)
date1 = datetime.date(2019, 8, 1)
date2 = datetime.date(2019, 8, 30)
curs_period = get_curs_period(date1, date2, valute_code)
print(f'Курсы валюты {valute} с даты {date1} по дату {date2} :')
print('Дата        Курс в руб.')
for item in curs_period:
    print(f"{item['Date'].date()}  {item['Curs']}")

Forcing soap:address location to HTTPS
Forcing soap:address location to HTTPS


Курсы валюты USD с даты 2019-08-01 по дату 2019-08-30 :
Дата        Курс в руб.
2019-08-01  63.4172
2019-08-02  63.834
2019-08-03  64.6423
2019-08-06  65.0546
2019-08-07  65.203
2019-08-08  65.0932
2019-08-09  65.1299
2019-08-10  65.2543
2019-08-13  65.4287
2019-08-14  65.6017
2019-08-15  65.2032
2019-08-16  65.8907
2019-08-17  65.9961
2019-08-20  66.6082
2019-08-21  66.784
2019-08-22  66.263
2019-08-23  65.6196
2019-08-24  65.6046
2019-08-27  65.9735
2019-08-28  66.2608
2019-08-29  66.4127
2019-08-30  66.7471


In [246]:
mongo_client = MongoClient('mongodb://127.0.0.1:27017')
db = mongo_client['curs_db']
curs_db = db.curs_db

In [247]:
curs_db.drop()
curs_db.insert_many(curs_period)

2. Написать функцию, которая принимает в качестве параметров две даты, ищет самую большую разницу между курсами валюты за указанный период и возвращает эти дни.

In [248]:
def find_min_max_period(date1, date2):
    datemin, datemax = curs_db.find({'Date': {'$gt':date1}}).sort('Curs',)[0]['Date'], \
                        curs_db.find({'Date': {'$lt':date2}}).sort('Curs', -1)[0]['Date']
    return datemin, datemax

date1 = datetime.datetime(2019, 8, 20, 0, 0, tzinfo=datetime.timezone.utc)
date2 = datetime.datetime(2019, 8, 30, 0, 0, tzinfo=datetime.timezone.utc)
datemin, datemax = find_min_max_period(date1, date2)
cursmin, cursmax = curs_db.find_one({'Date': datemin})['Curs'], curs_db.find_one({'Date': datemax})['Curs'] 
print(date1.date(), date2.date())
print(datemin.date(), datemax.date())
print(cursmin, cursmax)

2019-08-20 2019-08-30
2019-08-23 2019-08-20
65.6046 66.784


3. Сделать вывод информативным: Валюту <Валюта> выгодно было купить <Дата>, а продать <Дата>. Прибыль: <Разница в цене валюты>

In [250]:
print(f'Валюту {valute} выгодно было купить {datemin.date()}, а продать {datemax.date()}. Прибыль: {cursmax-cursmin:.4} руб.')

Валюту USD выгодно было купить 2019-08-23, а продать 2019-08-20. Прибыль: 1.179 руб.


4. Доработать приложение по вакансиям. При наличии вакансии с зарплатой отличной от руб. приложение должно осущетсвлять автоматический перевод по курсу валюты на сегодняшний день. Информацию получать через SOAP-сервис центробанка.